In [1]:
library(e1071)
library(tm)

Loading required package: NLP



## P3

In [19]:
smsdata <- read.csv("smsspamcollection/SMSSpamCollection",sep="\t",header=FALSE,quote="",stringsAsFactors=FALSE)
names(smsdata) <- c("Class","Message")

In [25]:
smscorpus <- Corpus(VectorSource(as.vector(smsdata$Message)))

In [28]:
# convert to lower case
smscorpus <- tm_map(smscorpus, content_transformer(tolower))
# remove stopwords
smscorpus <- tm_map(smscorpus, removeWords, stopwords("english"))
# remove whitespace
smscorpus <- tm_map(smscorpus, stripWhitespace)
# remove punctuation
smscorpus <- tm_map(smscorpus, removePunctuation)

Warning message in tm_map.SimpleCorpus(smscorpus, content_transformer(tolower)):
"transformation drops documents"
Warning message in tm_map.SimpleCorpus(smscorpus, removeWords, stopwords("english")):
"transformation drops documents"
Warning message in tm_map.SimpleCorpus(smscorpus, stripWhitespace):
"transformation drops documents"
Warning message in tm_map.SimpleCorpus(smscorpus, removePunctuation):
"transformation drops documents"


In [60]:
# create DocumentTermMatrix
smsmat <- DocumentTermMatrix(smscorpus)
# find frequent terms
smsfreq <- findFreqTerms(smsmat, 10)
# # create matrix with the frequent terms
smsmat <- smsmat[,smsfreq]
# smsmat <- smsmat[,smsfreq]
# train test split
trainIdx <- createDataPartition(
  1:dim(smsmat)[1],
  times = 1,
  p = 0.8,
  list = T)$Resample1
smsmatTrain <- smsmat[trainIdx,]
smsmatTest <- smsmat[-trainIdx,]
classTrain <- smsdata$Class[trainIdx]
classTest <- smsdata$Class[-trainIdx]

In [65]:
# convert the DocumentTermMatrix train/test matrices to a Boolean representation
smsboolTrain <- as.matrix(smsmatTrain)
smsboolTrain[smsboolTrain > 0] = 1
smsboolTest <- as.matrix(smsmatTest)
smsboolTest[smsboolTest > 0] = 1

In [83]:
# fit a SVM model
smsTrain <- data.frame(smsboolTrain)
smsTrain$Class <- as.factor(classTrain)
svmfit <- svm(Class~ ., data=smsTrain, kernel = "linear", scale = FALSE)

In [89]:
# training accuracy
train.pred <- predict(svmfit, smsTrain[,-length(smsTrain)])
sum(train.pred==classTrain)/length(classTrain)

[1] 0.9952936

In [90]:
# test accuracy
test.pred <- predict(svmfit, newdata=data.frame(smsboolTest))
sum(test.pred==classTest)/length(classTest)

[1] 0.9802158